In [ ]:
import warnings
import astropy.units as u
import ipyaladin.aladin_widget as ipal
import matplotlib.pyplot as plt
from style import (highlight_dec_cut, highlight_visibility, 
                   insert_tns_links_into_df,
                   insert_survey_links_into_df,
                   highlight_gal_lat, get_styled_html_table,
                   highlight_general_traffic_light)
from tns_downloader import TNSDownloader
from filter_targets import TargetFilter
from visibility import insert_max_alt_in_frame, insert_gal_lat_in_frame
from visualization import get_aladin_coords
from IPython.display import (display, HTML, display_html,
                             Markdown, Latex)
from ipywidgets import (Box, HBox, VBox, Label, Layout,
                        ToggleButtons, FloatSlider, widgets,
                        interactive, Button)
from ipywidgets import HTML as HTML_widget
from astropy.coordinates import SkyCoord
from astropy.table import Table
from visibility import Visibility
from redshift import GladeRedshiftCatalogue
from ob_generation import OBGenerator
from functools import partial

In [ ]:
# Uncomment this to parse data from the TNS

#tns_request = TNSDownloader(number_of_days=2)
#targets = insert_max_alt_in_frame(tns_request.result_compact)
#targets.to_hdf('data/targets.hdf', 'targets')

targets = pd.HDFStore('data/targets.hdf', mode='r')['targets']

# Classification targets

In [ ]:
targets, names = insert_tns_links_into_df(targets)
targets = insert_gal_lat_in_frame(targets)
targets = insert_survey_links_into_df(targets)
targets.index = names

In [ ]:
rsc = GladeRedshiftCatalogue('redshift_data/glade_v2.3.hdf')

In [ ]:
obj_type = ToggleButtons(
    options=['unclassified', 'SN II', 'SN IIP', 'SN Ia'],
    description='Type:',
    disabled=False,
    button_style=''
)
max_alt = FloatSlider(
    value=30.,
    min=0.,
    max=90.,
    step=2.,
    description='Max alt cut:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
mag_cut = FloatSlider(
    value=17.,
    min=10.,
    max=22.,
    step=0.25,
    description='Max mag cut:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

tf = TargetFilter(targets)
targets_filtered = interactive(tf.filter_targets, obj_type=obj_type,
                               max_alt=max_alt, mag_cut=mag_cut)
#targets_filtered.layout.flex_flow = 'column'
#targets_filtered.layout.border='solid'

display(targets_filtered)

In [ ]:
highlight_z = partial(highlight_general_traffic_light, levels=[0.03, 0.04, 0.12])
highlight_abs_mag = partial(highlight_general_traffic_light, levels=[-18.0, -17.5, 1e4])

In [ ]:
%matplotlib widget
plt.ioff()


checks = []
aladins = []
tables = []

layout_aladin = Layout(width='40.00%')
layout_aladin.margin = '87.5px 0px 0px 75px'

layout = Layout(width='100%')
layout.margin = '100px px 0px 0px'

for i in range(len(targets_filtered.result)):
    widgets = []

    df_object = pd.DataFrame(targets_filtered.result.iloc[i].values[np.newaxis,:],
                             columns=targets.columns)
    
    row = targets_filtered.result.iloc[i]
    coord = SkyCoord(row.RA, row.DEC, unit=(u.hourangle, u.deg))
    
    label = Label(r'\(\color{red}{' + row.name + '}\)')
    
    # Aladin widgets
    
    if coord.dec < -30. * u.deg:
        survey = 'P/DSS2/color'  # 'P/SDSS9/color'
    else:
        survey = 'P/PanSTARRS/DR1/color-z-zg-g'
    aladin = ipal.Aladin(target=get_aladin_coords(targets_filtered.result, i),
                         layout=layout_aladin, fov=0.0333,
                         survey=survey)
    
    # Visibility
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        vis_fig = Visibility(skycoord=coord).plot()
    
    # Redshift
    m = rsc.find_host_z_from_row(row, max_dist_kpc=40.)
    
    if len(m) > 0:
        check = Button(description="Generate OB!")
        obgen = OBGenerator.from_row(row)
        check.on_click(obgen)

        widgets.append(HBox([label, check]))
        box = HBox([aladin, vis_fig.canvas], layout=layout)
        widgets.append(box)

        vbox = VBox(widgets, layout=Layout(width='100%'))

        display(vbox)

        checks.append(check)

        display(HTML(get_styled_html_table(df_object)))
        display(HTML(m[rsc.comp_repr].style.apply(
            highlight_z, subset=['z']).apply(highlight_abs_mag, subset=['abs_mag']
                                            ).hide_index().set_caption(
                                                    'Possible hosts').render()))
        table = rsc.generate_aladin_table(m)
        aladins.append(aladin)
        tables.append(table)
    else:
        continue

    
    display(HTML('<hr />' + '-' * 200))

# Add host galaxies to aladin widget
for aladin, table in zip(aladins, tables):
    aladin.add_table(table)

## TODO:

* highlight spectroscopic redshifts
* add 2df redshift data, SDSS?